In [1]:
from sompz.functions_sompz import *
from sompz.functions_WL_Y3 import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py

## Files and global variables

In [2]:
deep_som_size = 64*64 
wide_som_size = 32*32

n_bins = 4
bin_edges =  [0.0, 0.358, 0.631, 0.872, 2.0] 

data_dir = './output_y3data/'
run_name = 'SOMF'

balrog_file = '/global/cscratch1/sd/acampos/sompz_data/v0.50_andresa/deep_balrog.pkl'
deep_cells_assignment_balrog_file = '/global/cscratch1/sd/acampos/sompz/test/full_run_on_data/SOM/cats_Y3/cells_deep_balrog_newSOM_y3.txt'
wide_cells_assignment_balrog_file = '/global/cscratch1/sd/acampos/sompz/test/full_run_on_data/SOM/cats_Y3/cells_wide_balrog_SOMF_230117.txt'

smooth_response_filename = '/global/u1/d/dgruen/work/sompz/test/full_run_on_data/y3_shape_response_grid_03_31_20.txt'

cosmos_file = '/global/cscratch1/sd/jmyles/COSMOS2015/sompz_cosmos.h5'

wide_field_file = '/global/cscratch1/sd/acampos/sompz_data/Y3_mastercat_03_31_20.h5'
wide_cells_assignment_file = '/global/cscratch1/sd/acampos/sompz/test/full_run_on_data/SOM/cats_Y3/cells_wide_assignment_all.csv'

In [3]:
# settings for output histogram - note that zbins_max needs to agree with Laigle currently
zbins_max = 6.00
zbins_dz  = 0.01

# all z-bins above zmax_pileup will be piled into this last bin
# Can up or downweight the last bin
zmax_pileup = 3.0
zmax_weight = 1.0

In [4]:
zbins  = np.arange(-zbins_dz/2.,zbins_max+zbins_dz,zbins_dz) # need to match Laigle binning if in 'usepz' mode
zmean  = (zbins[1:] + zbins[:-1])/2
fullpz = ["Z{:.2f}".format(s).replace(".","_") for s in np.arange(0,6.01,0.01)] # stays hardcoded while we use Laigle catalog as only truth

In [5]:
key      = fullpz
pointz   = 'Z'
keylabel = 'modal_even'

## Load cell assignment into dataframes for each dataset and calculate weights

In [6]:
balrog_data = build_balrog_df(balrog_file, 
                    deep_cells_assignment_balrog_file, 
                    wide_cells_assignment_balrog_file)

balrog_data['overlap_weight'] = calculate_weights(smooth_response_filename, 
                                   balrog_data['unsheared/snr'], 
                                   balrog_data['unsheared/size_ratio'], 
                                   balrog_data["injection_counts"], 
                                   balrog_data['unsheared/weight'], 
                                   len(balrog_data))

Length of balrog_data: 2417437


In [7]:
spec_data = build_spec_df(cosmos_file, balrog_data)

/global/homes/a/acampos/.conda/envs/mySOM/lib/python3.9/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
/global/homes/a/acampos/.conda/envs/mySOM/lib/python3.9/site-packages/pandas/core/indexing.py:1678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = empty_value


n duplicated Laigle 39399
all cosmos deep:  521851
matched cosmos deep:  451766
unmatched cosmos deep:  70085


In [8]:
wide_data_assignment = pd.read_csv(wide_cells_assignment_file, index_col = 0, dtype =np.int32)

wide_data = build_wide_df(wide_field_file, wide_data_assignment)  

wide_data['overlap_weight'] = calculate_wide_overlap_weight(wide_data['unsheared/R11'], 
                                                    wide_data['unsheared/R22'], 
                                                    wide_data['unsheared/weight'])

/global/homes/a/acampos/.conda/envs/mySOM/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


read select metacal done
read coadd_object_id done
read unsheared/T done
read unsheared/snr done
read unsheared/R11 done
read unsheared/R22 done
read unsheared/wight done


## Compute p(chat), p(c | chat), p(z | c) and p(z | chat)

In [9]:
cells, cell_weights_wide = get_cell_weights_wide(wide_data, 
                                                 overlap_weighted_pchat=True, 
                                                 force_assignment=False, 
                                                 cell_key='cell_wide_unsheared')

np.save(data_dir + 'pchat.npy', cell_weights_wide)

In [10]:
pcchat = calculate_pcchat(deep_som_size, 
                          wide_som_size,
                          balrog_data['cell_deep'],
                          balrog_data['cell_wide_unsheared'], 
                          balrog_data['overlap_weight'])

np.save(data_dir + 'pcchat.npy', pcchat)

/global/homes/a/acampos/.conda/envs/mySOM/lib/python3.9/site-packages/pandas/core/arraylike.py:364: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
all_deep_cells = np.arange(deep_som_size)

pz_c = np.array(get_deep_histograms(wide_data,
                                    spec_data,
                                    key=key,
                                    cells=all_deep_cells,
                                    overlap_weighted_pzc=True,
                                    bins=zbins))
np.save(data_dir + 'pzc.npy', pz_c)

In [12]:
all_wide_cells = np.arange(wide_som_size)

pz_chat = np.array(histogram(wide_data,
                             spec_data,
                             key=key,
                             pcchat = pcchat,
                             cells=all_wide_cells, 
                             cell_weights=np.ones(len(all_wide_cells)), 
                             overlap_weighted_pzc=True,
                             bins=zbins, 
                             individual_chat=True))

np.save(data_dir + 'pzchat.npy', pz_chat)

## Compute N(z)s 

In [15]:
# redshift data into bins

tomo_bins_wide_dict = bin_assignment_spec(spec_data,
                                          deep_som_size,
                                          wide_som_size,
                                          inj_counts = balrog_data['injection_counts'],
                                          inj_ids = balrog_data['true_id'],
                                          bin_edges = bin_edges)

# convert to format where tomo_bins_wide[i] is a 2D array, first column cell_id, second column an arbitrary weight
# tomo_bins_wide = tomo_bins_wide_2d(tomo_bins_wide_dict)

In [16]:
# wide data bin occupation

cell_occupation_info = wide_data.groupby('cell_wide_unsheared')['cell_wide_unsheared'].count()
# pickle.dump(cell_occupation_info, open('cell_occupation_info_wide_som_SOMF_230117.pkl', 'wb'))

bin_occupation_info = {'bin' + str(i) : np.sum(cell_occupation_info.loc[tomo_bins_wide_dict[i]].values) for i in range(n_bins)}
print(bin_occupation_info)

{'bin0': 23580877, 'bin1': 27236473, 'bin2': 24773059, 'bin3': 24618535}


In [17]:
tomo_bins_wide = tomo_bins_wide_2d(tomo_bins_wide_dict)

In [18]:
# NOT_BIN_CONDITIONALIZED

hists_wide = redshift_distributions_wide(data = wide_data,
                                         deep_data = spec_data,
                                         overlap_weighted_pchat = True, 
                                         overlap_weighted_pzc = True, 
                                         bins = zbins,
                                         pcchat = pcchat,
                                         tomo_bins = tomo_bins_wide,
                                         key = key,
                                         force_assignment = False,
                                         cell_key = 'cell_wide_unsheared')

np.save(data_dir+'Y3_hists_wide_NOT_BIN_CONDITIONALIZED_{}.npy'.format(keylabel), hists_wide)
plot_nz(hists_wide, zbins, data_dir+'Y3_nz_newbinning_onwide_NOT_BIN_CONDITIONALIZED_{}.png'.format(keylabel))

In [19]:
# bin conditionalized n(z)

hists_wide_bin_cond = np.array([nz_bin_conditioned(wide_data, 
                                                   spec_data, 
                                                   overlap_weighted_pchat= True, 
                                                   overlap_weighted_pzc=True, 
                                                   tomo_cells=tomo_bins_wide[i], 
                                                   zbins=zbins, 
                                                   pcchat = pcchat, 
                                                   cell_wide_key='cell_wide_unsheared', 
                                                   zkey=key) for i in range(n_bins)])

np.save(data_dir + 'Y3_hists_wide_bin_conditionalized_{}.npy'.format(keylabel), hists_wide_bin_cond)
plot_nz(hists_wide_bin_cond, zbins, data_dir + 'Y3_nz_newbinning_onwide_bin_cond_{}.png'.format(keylabel))
plot_nz_overlap([hists_wide_bin_cond], [keylabel], data_dir)

full redshift sample: 451766
subset of redshift sample in bin: 104053
full redshift sample: 451766
subset of redshift sample in bin: 122062
full redshift sample: 451766
subset of redshift sample in bin: 115420
full redshift sample: 451766
subset of redshift sample in bin: 110231


No handles with labels found to put in legend.


## Pile up very high z

In [20]:
zbins_piled, zmean_piled, hists_wide_bin_cond_piled = pileup(hists_wide_bin_cond,
                                                             zbins,
                                                             zmean,
                                                             zmax_pileup,
                                                             zbins_dz,
                                                             zmax_weight,
                                                             n_bins)

means_bc_piled, sigmas_bc_piled = get_mean_sigma(zmean_piled, hists_wide_bin_cond_piled)

np.save(data_dir + 'Y3_hists_wide_bin_conditionalized_pileup3_{}.npy'.format(keylabel), hists_wide_bin_cond_piled)
plot_nz(hists_wide_bin_cond_piled, zbins_piled, data_dir+'Y3_nz_newbinning_onwide_bincond_pilledup_{}.png'.format(keylabel))


save_des_nz(hists_wide_bin_cond_piled, 
            zbins_piled, 
            n_bins, 
            data_dir, 
            run_name, 
            keylabel+'_Y3_bincond_pileup3') ##THIS IS THE NZ WE USE

Dz, new-end-z,weight:  0.01 3.0 1.0
mkdir -p ./output_y3data/
write ./output_y3data/Y3_y3_redshift_distributions_SOMF_modal_even_Y3_bincond_pileup3.fits


## Smooth

In [23]:
# write nzs into an existing 2pts fits file 

template_fits='/global/cscratch1/sd/acampos/cosmosis/cosmosis-des-library/y3-3x2pt/data/des-y3/sim_data_vectors/sim_3x2pt_TATT_sample3.fits'
file_nzs = data_dir+f'Y3_y3_redshift_distributions_{run_name}_modal_even_Y3_bincond_pileup3.fits'
out_file_non_smooth = data_dir + f'Y3_2pt_{keylabel}_{run_name}_non_smooth.fits'

to2point(out_file_non_smooth,
         file_nzs,
         template_fits,
         run_name,
         keylabel,
         data_dir)

In [24]:
####################
# Smooth (is this just for plotting?)


file_nzs_smooth = data_dir + f'Y3_y3_redshift_distributions_{keylabel}_{run_name}_bincond_pileup3_smooth.txt'
out_file_smooth = data_dir + f'Y3_2pt_{keylabel}_{run_name}_smooth.fits'

smooth(out_file_smooth,
       out_file_non_smooth,
       file_nzs_smooth,
       run_name,
       keylabel,
       data_dir,
       hists_wide_bin_cond_piled)

#END smooth

## Generate h5 file

In [26]:
# convert to DF with cell <-> bin relation
tmp_cells = np.concatenate([tomo_bins_wide[nbin][:,0] for nbin in tomo_bins_wide])
tmp_bins = np.concatenate([(np.ones(len(tomo_bins_wide[nbin][:,0])) * nbin).astype(int) for 
                           nbin in tomo_bins_wide])
tomo_bin_hashtable = pd.Series(tmp_bins, tmp_cells)

nrows = wide_data['coadd_object_id'].shape[0]

In [28]:
f = h5py.File('sompz_test.hdf5','w', track_order=True)

In [29]:
fluxtypes = ['unsheared', 'sheared_1m', 'sheared_1p', 'sheared_2m', 'sheared_2p']

# add binning info to catalog
for fluxtype in fluxtypes:
    if fluxtype == 'unsheared':
        f.create_dataset('catalog/sompz/unsheared/coadd_object_id', data = wide_data['coadd_object_id'])
    
    print('add binning {}'.format(fluxtype))
    try:
        #print(f['catalog/sompz/unsheared'].keys())
        f.create_dataset(f'catalog/sompz/{fluxtype}/bhat', 
                         maxshape=(nrows,),
                         shape=(nrows,), dtype='i8')
        print("that worked!")
    except Exception as e:
        print(e)
    tmp = wide_data['cell_wide_{}'.format(fluxtype)].map(tomo_bin_hashtable)
    tmp[np.isnan(tmp)] = -1
    f['catalog/sompz/{}/bhat'.format(fluxtype)][...] = tmp
    f.create_dataset(f'catalog/sompz/{fluxtype}/cell_wide', data = wide_data[f'cell_wide_{fluxtype}'])
    


add binning unsheared
that worked!
add binning sheared_1m
that worked!
add binning sheared_1p
that worked!
add binning sheared_2m
that worked!
add binning sheared_2p
that worked!


In [30]:
f.create_dataset('catalog/sompz/pzdata/pz_c', data = pz_c)
print(f['catalog/sompz/pzdata/pz_c'][...].shape)

(4096, 601)


In [31]:
f.create_dataset('catalog/sompz/pzdata/pz_chat', data = pz_chat)
print(f['catalog/sompz/pzdata/pz_chat'][...].shape)

(1024, 601)


In [33]:
nz = fitsio.read(data_dir+'Y3_y3_redshift_distributions_SOMF_modal_even_Y3_bincond_pileup3.fits')

f.create_dataset('catalog/sompz/pzdata/bin0',  data = nz['BIN1'])
f.create_dataset('catalog/sompz/pzdata/bin1',  data = nz['BIN2'])
f.create_dataset('catalog/sompz/pzdata/bin2',  data = nz['BIN3'])
f.create_dataset('catalog/sompz/pzdata/bin3',  data = nz['BIN4'])
f.create_dataset('catalog/sompz/pzdata/zhigh', data = nz['Z_HIGH'])
f.create_dataset('catalog/sompz/pzdata/zlow',  data = nz['Z_LOW'])

<HDF5 dataset "zlow": shape (300,), type ">f8">

In [34]:
f.close()